In [3]:
import pandas as pd
import numpy as np
import ROOT

data = pd.read_csv("/eos/user/w/wrsmith/SWAN_projects/Data/DOF_P_Oct17_5_1_20Nov_stripid.txt",  sep=r'\t', engine='python')

EE = data[(data.CMSSWID > 872000000)]
EB = data[(data.CMSSWID < 872000000)]


#Decimal weights to encoded weights8
def decimal_to_encoded(weights):
    enc_weights = [0]*5
    count = -1
    for w in weights:
        count = count+1
        if w >0:
            encodedw = w*64
            encodedw = round(encodedw, 0)
            encodedw = int(encodedw)
            enc_weights[count] = encodedw
        else:
            encodedw = w*64
            encodedw = abs(encodedw)
            encodedw = round(encodedw, 0)
            encodedw = 128 - encodedw
            encodedw = int(encodedw)
            enc_weights[count] = encodedw
    return enc_weights
    
#Encoded weights back to decimals to look at differences
def encoded_to_decimal(enc_weights):
    back_weights = [0]*5
    count = -1
    for encw in enc_weights:
        count = count +1
        if encw > 64:
            encw = -(128 -encw)
        back_weights[count] = encw/64    
    return back_weights

#Performs a correction to encoded weights that don't sum to 256
def correction(enc_weights_not_corrected, back_weights):
    
    diff = 256 - sum(enc_weights_not_corrected)
    weightdiff = np.subtract(weights, back_weights)
    #print(weightdiff)
    
    if diff>0 and diff <6:
        pos = np.argsort(weightdiff)[-diff:]
        for k in pos:
            enc_weights_not_corrected[k] = enc_weights_not_corrected[k] + 1   
    elif diff< 0 and diff> -6:
        d = abs(diff)
        pos = np.argsort(weightdiff)[:d]
        
        #print(pos)
        for k in pos:
            enc_weights_not_corrected[k] = enc_weights_not_corrected[k] - 1   
                       
    else:

        enc_weights_not_corrected = 0
        print("\nError. The sum of the encoded weights differ from 256 by more than 5. Needs more investigation- dodgy numbers?")
        print("The encoded weights for this XTAL have been set to 0")
    
    return enc_weights_not_corrected
        

#-------------EE------------------------


#Set to 1 for stats about the weights before and after encoding
describe = 1

if describe ==1:
    print("Data shape ", EE.shape)    
    print("\nW1 described:\n", EE['w3'].mean())
    print("\nW2 described:\n", EE['w4'].mean())
    print("\nW3 described:\n", EE['w5'].mean())
    print("\nW4 described:\n", EE['w6'].mean())
    print("\nW5 described:\n", EE['w7'].mean())
    
weights = [EE['w3'].mean(), EE['w4'].mean(), EE['w5'].mean(), EE['w6'].mean(), EE['w7'].mean()]

#Encodes the perfect weights
enc_weights_not_corrected = decimal_to_encoded(weights)
    
#Calculates the decimal weights from the encoded ones - result will be one of the 1/64 allowed weights
back_weights = encoded_to_decimal(enc_weights_not_corrected)
    
#Check if encoded weights sum =256. If not, correction is performed on the weight(s) that are furthest 
#from the perfect weights they correspond to.
    
if sum(enc_weights_not_corrected) != 256:
    enc_weights_corrected = correction(enc_weights_not_corrected, back_weights)
else:
    enc_weights_corrected = enc_weights_not_corrected

print("EE weights: ", enc_weights_corrected[0], enc_weights_corrected[1],enc_weights_corrected[2], enc_weights_corrected[3], enc_weights_corrected[4])

#-------EB-------

if describe ==1:
    print("Data shape ", EB.shape)    
    print("\nW1 described:\n", EE['w3'].mean())
    print("\nW2 described:\n", EB['w4'].mean())
    print("\nW3 described:\n", EB['w5'].mean())
    print("\nW4 described:\n", EB['w6'].mean())
    print("\nW5 described:\n", EB['w7'].mean())
    
weights = [EB['w3'].mean(), EB['w4'].mean(), EB['w5'].mean(), EB['w6'].mean(), EB['w7'].mean()]

#Encodes the perfect weights
enc_weights_not_corrected = decimal_to_encoded(weights)
    
#Calculates the decimal weights from the encoded ones - result will be one of the 1/64 allowed weights
back_weights = encoded_to_decimal(enc_weights_not_corrected)
    
#Check if encoded weights sum =256. If not, correction is performed on the weight(s) that are furthest 
#from the perfect weights they correspond to.
    
if sum(enc_weights_not_corrected) != 256:
    enc_weights_corrected = correction(enc_weights_not_corrected, back_weights)
else:
    enc_weights_corrected = enc_weights_not_corrected

print("EB weights: ", enc_weights_corrected[0], enc_weights_corrected[1],enc_weights_corrected[2], enc_weights_corrected[3], enc_weights_corrected[4])



Data shape  (14368, 24)

W1 described:
 -0.7331855752917412

W2 described:
 -0.4622420222773172

W3 described:
 0.33705167950419285

W4 described:
 0.513012191584101

W5 described:
 0.3453637264807644
EE weights:  81 98 22 33 22
Data shape  (60486, 24)

W1 described:
 -0.7331855752917412

W2 described:
 -0.5492280540232102

W3 described:
 0.25895833022083664

W4 described:
 0.48123981666418614

W5 described:
 0.3583872251015472
EB weights:  93 93 16 31 23
